First load the model

In [ ]:
import os
import numpy as np
import torch
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from transformers import AutoTokenizer,AutoModelForCausalLM
# model_path = "/proj/vondrick3/bigmodels/llama2_chat/converted_weights_llama_chat70b"

tokenizer = AutoTokenizer.from_pretrained(model_path, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

Get latent embeddings in regular forward pass

In [ ]:
prompt = '''
[INST] What is highest mountain in the world? [/INST]
'''

model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

outputs = model.forward_interpret(
                **model_inputs,
                return_dict=True,
                output_attentions=True,
                output_hidden_states=True,
            )
prompt_len = model_inputs['input_ids'].shape[-1]
prompt_tokenized = tokenizer.convert_ids_to_tokens(model_inputs['input_ids'][0])
prompt_tokenids = model_inputs['input_ids'][0]

[(idx, tokenizer.decode([i])) for idx, i in enumerate(prompt_tokenids)]

Specify intepretation prompt and replacing locations. Here, _ _ _ _ _ will be replaced. If you are using other interpretation prompts, you can check index of each token to decide replacing locations with

    [(idx, tokenizer.decode([i])) for idx, i in enumerate(repeat_prompt_tokenids)]

In [ ]:
repeat_prompts = "[INST] _ _ _ _ _ [/INST]\nSure, I'll summerize your message:"
start_insert = 4
end_insert = 9

model_inputs = tokenizer([repeat_prompt for i in range(len(batch_insert_infos))], return_tensors="pt").to("cuda:0")
repeat_prompt_n_tokens = model_inputs['input_ids'].shape[-1]

Set up `insert_info`.  

In [ ]:
all_insert_infos = []
for retrieve_layer in [0, 1, 5, 10,15,20,25,30,35,40,45,50,55,60,65,70,75,79, 80]:
    for retrieve_token in range(prompt_len):
        insert_info = {}
        insert_info['replacing_mode'] = 'normalized'
        insert_info['overlay_strength'] = 1
        insert_info['retrieve_layer'] = retrieve_layer
        insert_info['retrieve_token'] = retrieve_token
        for idx, layer in enumerate(model.model.layers):
            insert_locations = [i for i in range(start_insert, end_insert)]
            insert_info[idx] = (insert_locations, outputs['hidden_states'][retrieve_layer][0][retrieve_token].repeat(1,len(insert_locations), 1))
        all_insert_infos.append(insert_info)

Get interpretations by batch

In [ ]:
for batch_start_idx in range(0,len(all_insert_infos),bs):
    with torch.no_grad():

        batch_insert_infos = all_insert_infos[batch_start_idx:min(batch_start_idx+bs, len(all_insert_infos))]

        model_inputs = tokenizer([repeat_prompt for i in range(len(batch_insert_infos))], return_tensors="pt").to("cuda:0")
        repeat_prompt_n_tokens = model_inputs['input_ids'].shape[-1]

        output = model.generate_interpret(**model_inputs, max_new_tokens=30, insert_info=batch_insert_infos, pad_token_id=tokenizer.eos_token_id, output_attentions = False)

        cropped_interpretation_tokens = output[:,repeat_prompt_n_tokens:]
        batch_interpretations = tokenizer.batch_decode(cropped_interpretation_tokens, skip_special_tokens=True)
        


Here's another example to also get interpretation logits and relevancy score


In [ ]:
for batch_start_idx in range(0,len(all_insert_infos),bs):
    with torch.no_grad():

        batch_insert_infos = all_insert_infos[batch_start_idx:min(batch_start_idx+bs, len(all_insert_infos))]

        model_inputs = tokenizer([repeat_prompt for i in range(len(batch_insert_infos))], return_tensors="pt").to("cuda:0")
        repeat_prompt_n_tokens = model_inputs['input_ids'].shape[-1]

        output = model.generate_interpret(**model_inputs, max_new_tokens=30, insert_info=batch_insert_infos, pad_token_id=tokenizer.eos_token_id, output_attentions = False)

        cropped_interpretation_tokens = output[:,repeat_prompt_n_tokens:]
        batch_interpretations = tokenizer.batch_decode(cropped_interpretation_tokens, skip_special_tokens=True)
        

        
        original_attribution_outputs = model.forward_interpret(
                                                    output[...,:-1],
                                                    return_dict=True,
                                                    output_attentions=True,
                                                    output_hidden_states=True,
                                                    insert_info=batch_insert_infos,
                                                )

        original_final_logits = model.lm_head(original_attribution_outputs.hidden_states[-1])
        original_final_logits = torch.softmax(original_final_logits, dim=-1)

        corrupted_attribution_outputs = model.forward_interpret(
                        output[...,:-1],
                        return_dict=True,
                        output_attentions=True,
                        output_hidden_states=True,
                    )

        corrupted_final_logits = model.lm_head(corrupted_attribution_outputs.hidden_states[-1])
        corrupted_final_logits = torch.softmax(corrupted_final_logits, dim=-1)



        
        diff = (original_final_logits - corrupted_final_logits).abs() 

        indices = output[:,1:].detach().cpu().long()
        indices = indices.unsqueeze(-1)
        selected_diff = torch.gather(diff.detach().cpu(), 2, indices).squeeze(-1)

        original_probs = torch.gather(original_final_logits.detach().cpu(), 2, indices).squeeze(-1)
        cropped_original_probs = original_probs[:,repeat_prompt_n_tokens-1:]

        #interpretation logits
        interpretation_probs = cropped_original_probs#[:,:10].prod(dim=-1)

        #relevancy score
        cropped_selected_diff = selected_diff[:,repeat_prompt_n_tokens-1:]
        
